In [ ]:
import numpy as np
from mayavi import mlab
from scipy.ndimage import gaussian_filter
from scipy.sparse import load_npz
from tifffile import imread, imwrite
from tvtk.util import ctf
from matplotlib.pyplot import cm
from tvtk.util.ctf import ColorTransferFunction, PiecewiseFunction

# 3D rendering using Mayavi

This notebook is used to generate 3D images: the tectum mask rendering in Fig. 6**e** and the stacked imaging planes in Fig. 6**b**.

# 1. Rendering tectum mask

#### Loading relevant data

In [ ]:
stack = imread('/home/anleg84/Documents/Atlas/Mapzebrain/H2B-GCaMP6s.tif')
centroids = np.load('../Files/nodes_tectum_right_sampled.npy')
masks = load_npz('/home/anleg84/Documents/Atlas/Mapzebrain/' + 'region_masks.npz')

region_id = 22
mask_full = np.flip(np.reshape(masks[:, region_id].toarray(), [359, 974, 597], order='C'), axis=0)
mask_right = np.copy(mask_full)
mask_right[:, :, :284] = 0
mask = (mask_left > 0).astype('float')

mask_wholebrain = imread('/home/anleg84/GitHub/Legare-2024-zebrafish-brain-networks/Data/mask_atlas.tif')
mask_wholebrain = (mask_wholebrain > 0).astype('float')

mask_wholebrain_filtered = gaussian_filter(mask_wholebrain, (5, 5, 5))
mask_wholebrain_ = (mask_wholebrain_filtered > 0.49).astype('float')

#### Swapping stack axes to get proper projection angle

In [ ]:
stack = np.swapaxes(stack, 0, 2)
stack = np.flip(stack, axis=2)

mask_wholebrain = np.swapaxes(mask_wholebrain, 0, 2)
mask_wholebrain = np.flip(mask_wholebrain, axis=2)

mask = np.swapaxes(mask, 0, 2)
mask = np.flip(mask, axis=2)

#### Rendering scene

In [ ]:
mlab.options.offscreen = False

mlab.figure(bgcolor=(1, 1, 1), size=(1000, 1000))

# Whole-brain mask
contour_out = mlab.contour3d(mask_wholebrain, contours=[1.0], opacity=0.2, color=(1,1,1))

# Tectum mask
contour = mlab.contour3d(mask, contours=[1.0], opacity=0.5, color=(1,0,0))

# Tectum nodes
mlab.points3d(centroids[:, 0], centroids[:, 1], 359 - centroids[:, 2],
                  scale_factor=7,
                  color=(1, 0, 0),  # Red color for spheres
                  mode='sphere',
                  opacity=1.0)  # Adjust opacity for spheres

# Stack in background
cmap = 'binary'
vmin = 50
vmax = 255
src = mlab.pipeline.scalar_field(stack)
volume = mlab.pipeline.volume(src, vmin=vmin, vmax=vmax)

# Adjusting stack colormap
ctf = ColorTransferFunction()
values_array = np.linspace(vmin, vmax, 128)
values_cmap = np.linspace(0., 1, 128)
color_mapping = np.append(np.expand_dims(values_array, axis=1), cm.get_cmap(cmap)(values_cmap)[:, :3], axis=1)
for c in color_mapping:
    ctf.add_rgb_point(c[0], c[1], c[2], c[3])
volume._volume_property.set_color(ctf)
volume._ctf = ctf

# Adjusting stack opacity
otf = PiecewiseFunction()
otf.add_point(0, 0)
otf.add_point(vmin-1, 0)
otf.add_point(vmin, 0.01)
otf.add_point(vmax, 0.01)
volume._volume_property.set_scalar_opacity(otf)
volume.update_ctf = True

# Adjusting projection angle
mlab.view(azimuth=-135, elevation=60, distance=2000)

# Saving
mlab.savefig("../Figures/rendering_tectum.png", size=(2000, 2000))  
mlab.show()

# 2. Rendering piezo imaging planes

In [ ]:
image_stack = np.load('../Results/figure6_imaging_planes.npy')

Interactive view

In [ ]:
mlab.options.offscreen = False

mlab.figure(bgcolor=(1, 1, 1), size=(1000, 1000))

opacities = [0.25] * (image_stack.shape[0])
opacities[5] = 1

for z, image in enumerate(image_stack):
    x, y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
    mesh = mlab.mesh(x, y, np.ones_like(image) * 100 * -z, scalars=image, colormap="gray", opacity=opacities[z])

    vmin, vmax = 0, 15000
    mesh.module_manager.scalar_lut_manager.data_range = (vmin, vmax)

mlab.view(azimuth=45, elevation=60, distance=4000)

mlab.show()

Saving image

In [ ]:
mlab.options.offscreen = True

mlab.figure(bgcolor=(1, 1, 1), size=(1000, 1000))

opacities = [0.25] * (image_stack.shape[0])
opacities[5] = 1

for z, image in enumerate(image_stack):
    x, y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
    mesh = mlab.mesh(x, y, np.ones_like(image) * 100 * -z, scalars=image, colormap="gray", opacity=opacities[z])

    vmin, vmax = 0, 15000
    mesh.module_manager.scalar_lut_manager.data_range = (vmin, vmax)

mlab.view(azimuth=45, elevation=60, distance=4000)

mlab.savefig("../Figures/rendering_planes.png", size=(2000, 2000))  